# CMSC471 Artificial Intelligence - Fall 2021
# Instructor: Fereydoon Vafaei
# <font color=blue> Assignment-1: Solving Problems by Search </font>

*Type your name and ID here*

## Overview, Policies, and Learning Objectives

We've studied two types of search algorithms during the recent lectures: Uninformed Search and Informed Search.

Uninformed Search (aka blind search) refers to the search strategies that do not use additional information about states and goal of the search - nothing beyond the provided problem definition. The first algorithms that we studied from Chapter 3 of the Russel & Norvig textbook were Breadth First Search (BFS) and Depth First Search (DFS) which both fall into the category of Uniformed Search.

We also studied how we can modify DFS to combine the advantages of BFS (completeness and conditional optimality) with the main advantage of DFS --space complexity of $O(bm)$ where $b$ is the branching factor and $m$ is the maximum depth of the search space. Limited Depth Search and Iterative Deepening DFS were two algorithms that we studied in that section and you are going to use them in Part I of this assignment.

In Part II, you are going to practice Informed Search algorithms and heuristic functions.

Finally, in Part III, you will work on an example graph by applying A* and checking whether the provided heuristic is admissible and consistent or not.

Pedagogically, this assignment will help you:
- better understand how search algorithms are implemented and work in practice. 
- practice reading documentation. This is a very important skill in AI/ML/Data Science collaborative environments and teams.

<b>Very Important Note:</b> Read ALL the instructions in this notebook very carefully. **Wherever a link is provided, click on the link and read the resource!** Careless reading and skipping instructions are major sources of making mistakes and losing points in your assignments. Also note that this assignment has three parts and includes multiple steps and questions. You're strongly recommended to get started early and plan to finish well before the due date. Technical problems or other issues/questions on the due date or just a day before would NOT be accepted as an excuse for extension.

<b>Course Policy Reminder</b>
Debugging the codes and error resolution are ALWAYS the students' responsbility regardless of the source or the cause of the error. This policy will be enforced in email communications and the office hours. Keep in mind that all assignments are individual graded tasks. Any collaboration with other students is strictly prohibited and is considered as cheating. Students should NOT share any answer, solution, or code with other students. Violations of these policies would be penalized according to UMBC academic integrity policy.

**You must run ALL cells** and get the correct outputs for all cells and give complete answers to all questions. **Cells/codes with no output get zero!**

Follow the instructions for each step very carefully.

Wherever needed, you should replace `...` elipsis with your code.

`...` may indicate one or more lines of missing codes. Some outputs are provided to you to use as reference and to verify that your output is correct.

## Part I - Uninformed Search

In <b>Part I</b> of this assignment, you are going to apply Uninformed Search algorithms on a couple of search problems.

First, download the search notebook from AIMA GitHub repo. Make sure that it is the 4th edition:
https://github.com/aimacode/aima-python/blob/master/search4e.ipynb

You should use any of the classes and functions of `search4e.ipynb` that is needed to solve the problems. You can copy and paste the borrowed code or a modified version of the classes and function in `search4e.ipynb`. You can add cells in each Part where needed and import the codes.

### Import Cells

**Note:** In this assignment, students are required to import any necessary class or function from `search4e.ipynb` in the following cells, and add/modify the code wherever it is needed. You are required to specify via comment line at the top of an import cell to mark it as an import cell. You are also required to resolve any issues or errors that may arise to make all the imported codes run error-free. You can import all codes in one or multiple cells but make sure you import classes and function in the order that they are used.

In [9]:
# Import Cell

%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]


#Queue
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)


    
#Algorithms
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure



#routeProblem class
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5


class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=True):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

### Route Finidng Problem

Using `RouteProblem` and `Map` classes, define a few new problems as described below called `r5`, `r6`, `r7` that represent different routes on the following **directed** graph. Notice that the `Map` must be defined as **directed**.

**Hint**: See how Romania map is defined in `search4e.ipynb`; however, you don't need locations in this case. Also, all the costs are assumed to be identical (the default cost value is 1).

The `A1-graph` is illustrated below.

<img src="A1-graph.png" align="left"/>

In [10]:
A1_graph = Map(
    [('a','b'), ('a','c'), ('a','d'),
     ('b','e'), ('b','f'), ('b','g'),
     ('c','h'), ('c','i'),
     ('d','j'), ('d','k'),
     ('e','l'), ('e','m'),
     ('f','n'), ('f','o'),
     ('g','p'), ('g','q'), 
     ('l','r'), ('l','s'),
     ('r','t'), ('r','u'),
    ])

> `r5` should be defined from `a` to `m` <br>
> `r6` should be defined from `a` to `u` <br>
> `r7` should be defined from `b` to `d`

In [11]:
r5 = RouteProblem('a', 'm', map=A1_graph)

r6 = RouteProblem('a', 'u', map=A1_graph)

r7 = RouteProblem('b', 'd', map=A1_graph)

>Next, try DFS with some example tests on this graph. The correct output is provided for your reference.

In [12]:
path_states(depth_first_bfs(r5))

['a', 'b', 'e', 'm']

In [13]:
path_states(depth_first_bfs(r6))

['a', 'b', 'e', 'l', 'r', 'u']

>What if no path exists?! Let's try!

In [14]:
# Non-existing path!
path_states(depth_first_bfs(r7))

[]

>Next, try the tests on `r5` and `r6` with Depth-limited algorithm (with a max depth of 3) and Iterative-deepening search. Outputs are not provided for these tests.

In [15]:
path_states(depth_limited_search(r5, limit=3))

['a', 'b', 'e', 'm']

In [16]:
path_states(depth_limited_search(r6, limit=3))

[]

In [17]:
path_states(iterative_deepening_search(r5))

['a', 'b', 'e', 'm']

In [18]:
path_states(iterative_deepening_search(r6))

['a', 'b', 'e', 'l', 'r', 'u']

### Grid problem

Now, try Breadth-first search (`breadth_first_bfs`), DFS (`depth_first_bfs`), Depth-Limited with a default value of the limit, and Iterative Deepening on `d1` grid problem from `search4e.ipynb`.

You can use `path_states()` function for the grid problem too.

> **Note:** If any runs of the algorithms for this section (grid problem) generates error or takes more than 15 minutes on your machine, you may stop the kernel, leave a comment in your code indicating what happened, and move on. This is ONLY for this section (grid problem) of Part I. All other cells should have correct outputs with NO error.

In [19]:
# Import Cell

class GridProblem(Problem):
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(15, 30), goal=(130, 30), obstacles=(), **kwds):
        Problem.__init__(self, initial=initial, goal=goal, 
                         obstacles=set(obstacles) - {initial, goal}, **kwds)

    directions = [(-1, -1), (0, -1), (1, -1),
                  (-1, 0),           (1,  0),
                  (-1, +1), (0, +1), (1, +1)]
    
    def action_cost(self, s, action, s1): return straight_line_distance(s, s1)
    
    def h(self, node): return straight_line_distance(node.state, self.goal)
                  
    def result(self, state, action): 
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state
    
    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state
        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles
    
def random_lines(X=range(15, 130), Y=range(60), N=150, lengths=range(6, 12)):
    """The set of cells in N random lines of the given lengths."""
    result = set()
    for _ in range(N):
        x, y = random.choice(X), random.choice(Y)
        dx, dy = random.choice(((0, 1), (1, 0)))
        result |= line(x, y, dx, dy, random.choice(lengths))
    return result

def line(x, y, dx, dy, length):
    """A line of `length` cells starting at (x, y) and going in (dx, dy) direction."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

random.seed(42) # To make this reproducible

frame = line(-10, 20, 0, 1, 20) | line(150, 20, 0, 1, 20)
cup = line(102, 44, -1, 0, 15) | line(102, 20, -1, 0, 20) | line(102, 44, 0, -1, 24)


d1 = GridProblem(obstacles=random_lines(N=100) | frame)

In [20]:
path_states(breadth_first_bfs(d1))

[(15, 30),
 (16, 30),
 (17, 30),
 (18, 30),
 (19, 31),
 (20, 32),
 (21, 33),
 (22, 33),
 (23, 34),
 (24, 35),
 (25, 36),
 (26, 36),
 (27, 36),
 (28, 36),
 (29, 36),
 (30, 36),
 (31, 36),
 (32, 37),
 (33, 38),
 (34, 39),
 (35, 40),
 (36, 40),
 (37, 40),
 (38, 40),
 (39, 40),
 (40, 40),
 (41, 40),
 (42, 40),
 (43, 40),
 (44, 40),
 (45, 40),
 (46, 40),
 (47, 40),
 (48, 40),
 (49, 40),
 (50, 40),
 (51, 40),
 (52, 40),
 (53, 40),
 (54, 40),
 (55, 40),
 (56, 40),
 (57, 40),
 (58, 40),
 (59, 40),
 (60, 40),
 (61, 40),
 (62, 40),
 (63, 40),
 (64, 40),
 (65, 40),
 (66, 40),
 (67, 39),
 (68, 38),
 (69, 37),
 (70, 36),
 (71, 35),
 (72, 34),
 (73, 33),
 (74, 32),
 (75, 32),
 (76, 31),
 (77, 31),
 (78, 30),
 (79, 29),
 (80, 28),
 (81, 27),
 (82, 27),
 (83, 27),
 (84, 27),
 (85, 27),
 (86, 27),
 (87, 27),
 (88, 27),
 (89, 28),
 (90, 28),
 (91, 28),
 (92, 28),
 (93, 28),
 (94, 28),
 (95, 28),
 (96, 28),
 (97, 28),
 (98, 28),
 (99, 28),
 (100, 28),
 (101, 28),
 (102, 28),
 (103, 28),
 (104, 28),
 (105

In [46]:
path_states(depth_first_bfs(d1))
#Caused a max recursion depth exceeded error

RecursionError: maximum recursion depth exceeded

In [21]:
path_states(depth_limited_search(d1))

[]

In [ ]:
path_states(iterative_deepening_search(d1))
#Took greater than 15mins 

>Next, define a new grid problem named `d8` from `(4,4)` to `(8,3)` with no obstacles, and with the default directions, and run all the aforementioned algorithms BFS, DFS (`depth_first_recursive_search`), depth-limited with default limit and iterative-deepening on it.

In [24]:
d8 = GridProblem((4,4), (8,3))

In [25]:
path_states(breadth_first_bfs(d8))

[(4, 4), (5, 4), (6, 4), (7, 4), (8, 3)]

In [26]:
path_states(depth_first_recursive_search(d8))

[(4, 4), (5, 4), (6, 4), (7, 3), (8, 3)]

In [27]:
path_states(depth_limited_search(d8))

[(4, 4),
 (5, 3),
 (5, 2),
 (4, 1),
 (4, 0),
 (4, -1),
 (5, 0),
 (5, 1),
 (6, 2),
 (7, 2),
 (8, 3)]

In [28]:
path_states(iterative_deepening_search(d8))

[(4, 4), (5, 3), (6, 2), (7, 2), (8, 3)]

### Part I Questions

- **Q1** [2 points] - What is the minimum required depth limit for the **Depth-limited search** so that it can return the desired output `['a', 'b', 'e', 'l', 'r', 'u']` for `r6`? Why?


- **Q2** [4 points] - Which of the algorithms that you tried returned a solution path for `d1`?

- **Q3** [4 points] - Which of the algorithms that you tried returned a solution path for `d8`?

<font color=red>Enter your answers in the following markdown cell.</font>

- Your answers to Part I questions go here - below the line:

========================================================


YOUR Answers:

- Q1: 

The minimum required depth limit for depth-limited search in this case would be 5. Because depth limited search will limit its search to a specific depth level. And our goal of node u is in depth 5 of the tree, so for the search to ever reach node u our limit must be 5.

- Q2: 

Breadth_first_bfs returned a solution. The others either reached max recursion depth, took to long to calculate, or wasn't able to return a solution.

- Q3: 

breadth_first_bfs, depth_first_recursive_search, depth_limited_search, iterative_deepening_search all returned solutions. 

## Part II - Informed Search & Heuristics

In <b>Part II</b> of this assignment, you apply informed search algorithms on 8-puzzle problem.

Remember how I tried to [solve it online](http://www.tilepuzzles.com/default.asp?p=12) during the lecture? Now, Artificial Intelligence will help solve this puzzle! 

The state of the puzzle are represented as a list of integers. 0 represents the empty position. 

In [29]:
# Import Cell
import math
#8 Puzzle class
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h3(self, node):
        return math.sqrt(self.h2(node))
        
    def h4(self, node):
        return max(self.h1(node), self.h2(node))
    
    def h(self, node): return self.h2(node)
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

> The following is an example start state. The blank is represented by a 0 digit.

In [30]:
startState = (1, 0, 3, 4, 2, 5, 6, 7, 8)

> Define a new `EightPuzzle` problem named `e6` with `initial` as `startState`.

In [31]:
e6 = EightPuzzle(startState)

> Solve `e6` using `weighted_astar_search` and show the solution path with `path_states()` function.

In [32]:
for s in path_states(weighted_astar_search(e6)):
    print(board8(s))

1 _ 3
4 2 5
6 7 8

1 2 3
4 _ 5
6 7 8

1 2 3
_ 4 5
6 7 8

_ 2 3
1 4 5
6 7 8

2 _ 3
1 4 5
6 7 8

2 3 _
1 4 5
6 7 8

2 3 5
1 4 _
6 7 8

2 3 5
1 _ 4
6 7 8

2 _ 5
1 3 4
6 7 8

_ 2 5
1 3 4
6 7 8

1 2 5
_ 3 4
6 7 8

1 2 5
3 _ 4
6 7 8

1 2 5
3 4 _
6 7 8

1 2 _
3 4 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



### Unsolvable 8-Puzzle Problems

There are cases in which the [8 puzzle is not solvable](https://www.cs.princeton.edu/courses/archive/fall12/cos226/assignments/8puzzle.html). For example, in the simple example below, switching the position of 1 and 2 makes it impossible to reach the goal state. It would seem that this could be done with a few simple slides, but tests with several algorithms show that they cannot find a solution.

In general, an odd number of inversions ([inverted number positions](https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/)) from the starting state in comparision to the goal state leads to an unsolvable puzzle.

In [33]:
unsolvable_startState = (2, 1, 3, 4, 5, 6, 7, 8, 0)

> Try to define a new `EightPuzzle` problem using `unsolvable_startState`.

In [34]:
e7 = EightPuzzle(unsolvable_startState)

AssertionError: 

### Heuristic Functions: h3 and h4

Add two new heuristics `h3` and `h4` to `EightPuzzle` class. `h3` should be the squared root of Manhattan distance and `h4` should be defined as the maximum of `h1` and `h2`. Try running the `weighted_astar_search` on `e6` once with `h3` and once with `h4`. **They should run error-free. Your code will be tested!**

In [35]:
# With h3 heuristics 
for s in path_states(weighted_astar_search(e6, e6.h3)):
    print(board8(s))

1 _ 3
4 2 5
6 7 8

1 2 3
4 _ 5
6 7 8

1 2 3
_ 4 5
6 7 8

_ 2 3
1 4 5
6 7 8

2 _ 3
1 4 5
6 7 8

2 3 _
1 4 5
6 7 8

2 3 5
1 4 _
6 7 8

2 3 5
1 _ 4
6 7 8

2 _ 5
1 3 4
6 7 8

_ 2 5
1 3 4
6 7 8

1 2 5
_ 3 4
6 7 8

1 2 5
3 _ 4
6 7 8

1 2 5
3 4 _
6 7 8

1 2 _
3 4 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



In [36]:
# With h4 heuristics 
for s in path_states(weighted_astar_search(e6, e6.h4)):
    print(board8(s))

1 _ 3
4 2 5
6 7 8

1 2 3
4 _ 5
6 7 8

1 2 3
_ 4 5
6 7 8

_ 2 3
1 4 5
6 7 8

2 _ 3
1 4 5
6 7 8

2 3 _
1 4 5
6 7 8

2 3 5
1 4 _
6 7 8

2 3 5
1 _ 4
6 7 8

2 _ 5
1 3 4
6 7 8

_ 2 5
1 3 4
6 7 8

1 2 5
_ 3 4
6 7 8

1 2 5
3 _ 4
6 7 8

1 2 5
3 4 _
6 7 8

1 2 _
3 4 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



### Comparing Heuristics

In [37]:
# Import Cell
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))


Similar to **"Comparing heuritic"** report in `search4e.ipynb` and using `CountCalls` class, generate a report to compare the results of A* using the four heuristics [`h1`, `h2`, `h3`, `h4`] on `e6`.

In [38]:
def astar_misplaced_tiles(problem): 
    return astar_search(problem, problem.h1)

def astar_manhattan(problem): 
    return astar_search(problem, problem.h2)

def astar_sqrt_manhattan(problem): 
    return astar_search(problem, problem.h3)

def astar_max(problem): 
    return astar_search(problem, problem.h4)

report([astar_misplaced_tiles, astar_manhattan, astar_sqrt_manhattan, astar_max], 
       [e6])

astar_misplaced_tiles:
    1,341 nodes |      497 goal |   15 cost |     511 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
    1,341 nodes |      497 goal |   15 cost |     511 actions | TOTAL

astar_manhattan:
      448 nodes |      167 goal |   15 cost |     181 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      448 nodes |      167 goal |   15 cost |     181 actions | TOTAL

astar_sqrt_manhattan:
    4,115 nodes |    1,521 goal |   15 cost |   1,535 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
    4,115 nodes |    1,521 goal |   15 cost |   1,535 actions | TOTAL

astar_max:
      448 nodes |      167 goal |   15 cost |     181 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      448 nodes |      167 goal |   15 cost |     181 actions | TOTAL



### Part II Questions

- **Q4** [2 points] - Could you define `e7`? Explain why based on the provided resources in the links above.

- **Q5** [8 points] - According to the report you generated, which heuristic is the best one among `[h1, h2, h3, h4]`? Why? Explain COMPLETELY. Your explanation should be based on the report and the reasons discussed in Section 3.6.1 and Figure 3.26 of the textbook and should mention **Domination**.

- Your answers to Part II questions go here - below the line:

========================================================


YOUR Answers:

- Q4: 

No e7 could not be defined. 

In the 8 puzzle problem there is a concept of inversions, inversions being when a piece in the state is larger than a piece that follows it. The states of the 8-puzzle problem can be catagorized into two disjoined sets, one set containing states that have a even number of inversions, and the other containing states with an odd number of inversions. Its said that no matter what moves are done to the board, states with even number of inversions will remain even. and odd states will remain odd. So if the inital state of the problem had an odd number of inversions and the goal state has an even number then there is no way for the inital state to reach the goal state. 

The problem e7 has an odd number of inversions, while the goal state has an even number of inversions. Therefore the code determined that the e7 problem was unsolvable and did not bother trying to solve it. The reason for the inversions staying even or staying odd is because if you do a row move, the inversion count will not change, and if you do a column move the inversion count will change by 2 meaning even numbers will say even and odd will stay odd.


- Q5: 

The best heuristic among the four tested is the astar_manhattan. 

The reason being is that for any node, the h2 manhattan heuristic is always greater than or equal to the h1 misplaced tiles heuristic. This is considered to be Domination, in that h2 dominates h1, and A* using h2 will never expand more nodes than A* using h1. This is because every node that has f(n) < the cost of the optimal solution path, will be surely expanded when the heuristic is consistent. And since h2 is greater than or equal to h1, every node that is surely expanded by h2 is also in the set expanded by h1. but with h1 being a more of an under estimate it may cause more nodes to be expanded than h2 would. And thus being less efficient. 

You can see in the report that that all the heuristics found the cost optimal solution returning a cost of 15. But the number of nodes that the misplaced tiles and square root heuristic expanded is much higher that the manhattan heuristic. This is due to the misplaced tiles and the square root heuristic being smaller then the manatan heuristic, causeing it to under estimate more and expand more nodes. 

The astar_max heuristic managed to do just as well as the manhatan heuristic but thats only because it took the max between the manhatan and the misplaced tile heuristic. And we know that the manhatan heuristic will always be greater than the misplaced tile heuristic. So there is no point in using that, when you can directly use the manhatan heuristic. There for the manhatan heuristic is the best one among the 4 heuristics.      

## Part III - A*, Admissibility, and Consistency

The following graph shows the state space of a search problem. Answer the following questions.

<font color=red> **ALL your answers should be typed in including the math inequality. Handwritten answers or screenshots will get zero.**</font>

You MUST use [Latex](https://oeis.org/wiki/List_of_LaTeX_mathematical_symbols) for math. You can put your math equations between `$` You may also put them between `$$` to align it on center. See how Latex is used in the questions to display the math by putting them between `$$`

<img src="A-star.png" align="left"/>

### Part III Questions

- **Q6** [10 points] - Is this heuristic $h(n)$ admissible? You must prove admissibility using the inequality from slides for every node or to prove otherwise you must show a counter-example.
$$\forall\, node\,n, h(n) \le h^*(n)$$
> where $h^*(n)$ is the true actual (minimal) cost from $n$ to goal

- **Q7** [10 points] - Is this heuristic $h(n)$ consistent? You must prove consistency using the inequality from slides for every node or to prove otherwise you must show a counter-example.
 a heuristic $h$ is consistent if for every node $n$ of a parent node $p$,

$$h(p) \le h(n) + \mathrm{stepcost}(p,n)$$

- **Q8** [5 points] - Is **A\* Search** (tree version) guaranteed to find the optimal solution for this state space? Why? What solution and goal would be returned if you run **A\* Search** on this problem?

- Your answers to Part III questions go here - below the line:

========================================================


YOUR Answers:

- Q6- 

h(n) Is admissible.

For the goal G1, the paths from node S to G1 that can be taken and their actual costs are: 

S-A-C-F-G1 = 12

S-A-D-H-G1 = 12

S-A-D-J-G1 = 22

S-B-E-J-G1 = 21

Node S has $h(n) = 8$. so $h(n) \le h^*(n)$

The paths from node A to G1 that can be taken and their actual costs are: 

A-C-F-G1 = 8

A-D-H-G1 = 8

A-D-J-G1 = 18

Node A has $h(n) = 7$. so $h(n) \le h^*(n)$

The paths from node C to G1 that can be taken and their actual costs are: 

C-F-G1 = 5

Node C has $h(n) = 3$. so $h(n) \le h^*(n)$

The paths from node F to G1 that can be taken and their actual costs are: 

F-G1 = 4

Node F has $h(n) = 4$. so $h(n) \le h^*(n)$

The paths from node D to G1 that can be taken and their actual costs are: 

D-H-G1 = 3

D-J-G1 = 13

Node D has $h(n) = 2$. so $h(n) \le h^*(n)$

The paths from node H to G1 that can be taken and their actual costs are: 

H-G1 = 1

Node H has $h(n) = 1$. so $h(n) \le h^*(n)$

The paths from node J to G1 that can be taken and their actual costs are: 

J-G1 = 10

Node J has $h(n) = 5$. so $h(n) \le h^*(n)$

The paths from node B to G1 that can be taken and their actual costs are: 

B-E-J-G1 = 19

Node B has $h(n) = 6$. so $h(n) \le h^*(n)$

The paths from node E to G1 that can be taken and their actual costs are: 

E-J-G1 = 16

Node E has $h(n) = 3$. so $h(n) \le h^*(n)$


For the goal G2, The path from node S to G2 that can be taken and their actual costs are:

S-B-E-K-G2 = 8

Node S has $h(n) = 8$. so $h(n) \le h^*(n)$

The paths from node B to G1 that can be taken and their actual costs are: 

B-E-K-G2 = 6

Node B has $h(n) = 6$. so $h(n) \le h^*(n)$

The paths from node E to G1 that can be taken and their actual costs are: 

E-K-G2 = 3

Node E has $h(n) = 3$. so $h(n) \le h^*(n)$

The paths from node k to G1 that can be taken and their actual costs are: 

K-G2 = 2

Node K has $h(n) = 2$. so $h(n) \le h^*(n)$

So for this heuristic the equation: 

$$\forall\, node\,n, h(n) \le h^*(n)$$

Holds true and the heuristic is admissible.

 
- Q7-

The heuristic h(n) is not consistent.
h(n) is only consistent when 

$$h(p) \le h(n) + \mathrm{stepcost}(p,n)$$

But this heuristic can be proven that its not consistent with the following counter example involving the node A and C:

$$h(A) > h(C) + \mathrm{stepcost}(A,C)$$

The heuristic cost of node A is greater than the sum of the heuristic cost of node C and the step cost from A to C, which conflicts with the definition of a consistent heuristic, and therefore this heuristic is not consistent. 

- Q8-

If using the tree search version of A*, It only needs to have an admissible  heuristic to guarantee a optimal solution. This can be proven by contradiction. 

Lets assume that A* tree search with a admissible heuristic had a optimal path cost of $C'$ but the algorithm returned a non-optimal path with a cost of $C > C'$.

$$f(n) > C'$$

$$f(n) = g(n)+h(n)$$

This means there must be some node n which is on the optimal path but was not expaned. because A* should search all nodes on the optimal path, if it had done so then this node should have been expaned. 

Since the heuristic is admissible we know that the cost of the unexpaned node n to the goal must still be smaller than the true cost. This means there is still a valid unexpaned path to the goal that is smaller than the returnd cost C, and therefore also smaller than or equal to the true optimal cost. 

$$f(n) \le g'(n)+h'(n)$$

$$f(n) \le C'$$

where $g'(n)$ represent the cost of the optimal path from start to n and $h'(n)$ represent the cost of the optimal path from n to the nearest goal 

Since A* always picks the path with the most promising total cost to expand next and the heuristic is admissable, there is no way A* should have returned that non optimal solution. 

The solution returned for this probelm would be 

S-B-E-K-G2

ending up at goal G2. 



## Grading

Assignment-1 has a maximum of 100 points. Make sure that you get the desired outputs for all cells. Also, your notebook should be written with no grammatical and spelling errors and should be nicely-formatted and easy-to-read.

The breakdown of the 100 points is as follows:

Part I has 40 points:
- codes and correct outputs: [30 points]
- correct answer of the Part I questions: [10 points] {Q1: 2 points Q2 & Q3: 4 points each} 

Part II has 35 points:
- codes and correct outputs: [25 points]
- correct answers of the Part II questions: [10 points] {Q4: 2 points Q5: 8 points}

Part III has 25 points:
- Admissibility - Q6 [10 points] **Handwritten and non-LaTeX answers will get zero!**
- Consistency - Q7 [10 points] **Handwritten and non-LaTeX answers will get zero!**
- A* - Q8 [5 points]

<b>Note: </b>Follow the instructions of each section carefully. <b>Up to 10 points will be deducted if your submitted notebook is not easy to read and follow or if it has grammatical and spelling errors.</b>

Grading will be based on: 

  * correct implementation,
  * running ALL cells and correct outputs, (cells with no output get ZERO for the whole part that includes the cell except for the grid problem)
  * correct answer to the questions,
  * readability of the notebook.

## Submission

Name your completed notebook ```Lastname-A1.ipynb```. Submit the completed notebook using the ```Assignment-1``` link on Blackboard.
  
<font color=red><b>Due Date: Tuesday October 5th, 11:59PM</b></font>